In [1]:
import leafmap

In [2]:
# m = leafmap.Map(center=[51.512778, 7.465556], zoom=19, height="800px")
m = leafmap.Map(center=[39.133611, 117.205278], zoom=19, height="800px")
m.add_basemap("SATELLITE")
m

Map(center=[39.133611, 117.205278], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…

In [3]:
if m.user_roi is not None:
    bbox = m.user_roi_bounds()
else:
    bbox = [7.4, 51.433333, 7.633333, 51.600000]


In [4]:
import numpy as np
def clip_bbox(input_bbox, size_lat=0.01, size_lon=0.01):
    min_lon,min_lat,max_lon,max_lat=input_bbox[0],input_bbox[1],input_bbox[2],input_bbox[3]
    bbox_list=[]
    for i in np.arange(min_lon,max_lon,size_lon):
        for j in np.arange(min_lat,max_lat,size_lat):
            lat=min(size_lat+j,max_lat)
            lon=min(size_lon+i,max_lon)
            bbox_list.append([i,j,lon,lat])
    return bbox_list

bbox_list=clip_bbox(bbox)
print(len(bbox_list))    

408


In [6]:
# 单线程下载
for f in range(166,len(bbox_list)):
    if f>166:
        break
    image="./satellite/Dortmund_satellite_"+str(f)+".tif"
    leafmap.map_tiles_to_geotiff(
    output=image, bbox=bbox_list[f], zoom=20, source="Satellite", overwrite=True,quiet=True)
    print(f"TIFF file saved as {image}")


TIFF file saved as ./satellite/Dortmund_satellite_166.tif


In [ ]:
# 多线程下载
import concurrent.futures
def download_tile(tile_info):
    output, bbox, zoom, source = tile_info
    leafmap.map_tiles_to_geotiff(output=output, bbox=bbox, zoom=zoom, source=source,overwrite=True,quiet=True)
    print(f"TIFF file saved as {output}")

# 定义下载任务
tiles_info=[]
for f in range(140,len(bbox_list)):
    image="./satellite/Dortmund_satellite_"+str(f)+".tif"
    tiles_info.append((image,bbox_list[f],20,"Satellite"))
# print(tiles_info)

# 使用多线程下载
with concurrent.futures.ThreadPoolExecutor() as executor:
    executor.map(download_tile, tiles_info)

TIFF file saved as ./satellite/Dortmund_satellite_152.tif
TIFF file saved as ./satellite/Dortmund_satellite_163.tif
TIFF file saved as ./satellite/Dortmund_satellite_164.tif
TIFF file saved as ./satellite/Dortmund_satellite_165.tif
TIFF file saved as ./satellite/Dortmund_satellite_140.tif
TIFF file saved as ./satellite/Dortmund_satellite_167.tif
TIFF file saved as ./satellite/Dortmund_satellite_154.tif
TIFF file saved as ./satellite/Dortmund_satellite_151.tif
TIFF file saved as ./satellite/Dortmund_satellite_142.tif
TIFF file saved as ./satellite/Dortmund_satellite_158.tif
TIFF file saved as ./satellite/Dortmund_satellite_141.tif
TIFF file saved as ./satellite/Dortmund_satellite_144.tif
TIFF file saved as ./satellite/Dortmund_satellite_149.tif
TIFF file saved as ./satellite/Dortmund_satellite_146.tif
TIFF file saved as ./satellite/Dortmund_satellite_148.tif
TIFF file saved as ./satellite/Dortmund_satellite_150.tif
TIFF file saved as ./satellite/Dortmund_satellite_145.tif
TIFF file save

In [1]:
from samgeo import tms_to_geotiff, split_raster
import glob
import os
image_list= glob.glob("E:/satellite/*.tif")
for f in image_list:
    name = os.path.splitext(os.path.basename(f))[0]
    outdir = "./tiles/"+name
    split_raster(f, out_dir=outdir, tile_size=(1000, 1000), overlap=0)
    # 手动重命名输出文件
    for i, filename in enumerate(os.listdir(outdir)):
        os.rename(os.path.join(outdir, filename), os.path.join(outdir, name+'_'+filename))